In [1]:
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize
from scipy.optimize import curve_fit


#Alam=np.array([.2,.4])


#Tfunc = interpolate.interp1d(lam, Tdata)
#Gfunc = interpolate.interp1d(lam, Gdata)

#def func(x,d,b,c):
#    return d*Tfunc(x*(10**(c))-b*Gfunc(x))

#coef = np.zeros([len(Alam),3])
#for i in range(len(Alam)): 
#    coef[i,:]=scipy.optimize.curve_fit(func,lam,Odata)[0]
#    coef[i,2] = coef[i,2]/Alam[i]
#print(coef)


def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

#output1=cartesian_product(*[Gdata2, Tdata, Alam])


# Extinction Function

In [2]:
#Reddening law 

# Program to calculate an array of A's from an input wavelength array
# x has units of um^-1 
#Original name "mkafromlam"
#The extinction law is 'redreturn'


import numpy as np
Rv=3.1

#First create an array of zeros with the number of elements lambda has

def Alam(lamin):

    lam=np.zeros(len(lamin))
    invlammicro=np.zeros(len(lamin))
    redreturn=np.zeros(len(lamin))

    #Convert to microns, then invert to get invlammicro (x)
    for i in range(len(lam)):

        lam[i]=lamin[i]/10000.0
        invlammicro[i]=1/lam[i]

        
    #IR and Optical


    #For 0.3 < x < 1.1
        if 0.3 <= invlammicro[i]<=1.1:       
            a=0.574 * invlammicro[i] ** 1.61
            b=-0.527 * invlammicro[i] ** 1.61
            redreturn[i] = a + (b/Rv)

    #Optical, NIR


    #For  1.1 < x < 3.3    
        elif 1.1 < invlammicro[i] <= 3.3:
            y=invlammicro[i]-1.82
            a = 1.0 + 0.17699 * y - 0.50447 * y**2 - 0.02427 * y**3 + 0.72085 * y**4 + 0.01979 * y**5 - 0.77530 * y**6 +0.32999 * y**7
            b=1.41338 * y + 2.28305 * y**2 + 1.07233 * y**3 - 5.38434 * y**4 - 0.62251 * y**5 + 5.30260 * y**6 - 2.09002 * y**7
            redreturn[i] = a + (b/Rv)


    # UV and far-UV

    #For 3.3 < x < 5.9
        elif 3.3 < invlammicro[i] < 5.9:    
            a = 1.752 - 0.316 * invlammicro[i] - 0.104 / ((invlammicro[i]-4.67)**2 +0.341)
            b = -3.090 + 1.825 * invlammicro[i] + 1.206 / ((invlammicro[i] - 4.62)**2 + 0.263)
            redreturn[i] = a + (b/Rv)


    #For 5.9 < x < 8   
        elif 5.9 <= invlammicro[i] <= 8.0:
            Fa = -0.04473 * (invlammicro[i] - 5.9)**2 - 0.009779 * (invlammicro[i] - 5.9)**3
            Fb = 0.2130 * (invlammicro[i] - 5.9)**2 + 0.1207 * (invlammicro[i] - 5.9)**3
            a = 1.752 - 0.316 * invlammicro[i] - 0.104 / ((invlammicro[i]-4.67)**2 +0.341) + Fa
            b = -3.090 + 1.825 * invlammicro[i] + 1.206 / ((invlammicro[i] - 4.62)**2 + 0.263) + Fb
            redreturn[i] = a + (b/Rv)


    #For 8 < x < 10    
        elif 8.0 < invlammicro[i] <= 10.0:    
            a = -1.073 - 0.628*(invlammicro[i] - 8.0) + 0.137*(invlammicro[i] - 8.0)**2 - 0.070 * (invlammicro[i] - 8.0)**3
            b = 13.670 + 4.257*(invlammicro[i] - 8.0) - 0.420 * (invlammicro[i]-8.0)**2 + 0.374 * (invlammicro[i] - 8.0)**3
            redreturn[i] = a + (b/Rv)

    return redreturn

#print(len(redreturn))

In [3]:
print(Alam(np.array([5400])))

[1.02040436]


In [4]:
ls

Alam.ipynb         Chi squared.ipynb  gal/               sne/


In [5]:
import glob
from astropy import table
from astropy.io import ascii
from scipy import interpolate
import	multiprocessing as mp

## Reading in libraries

In [6]:
templates_gal = glob.glob('gal/**/*')
templates_gal = [x for x in templates_gal if 'CVS' not in x and 'README' not in x]
templates_gal = np.array(templates_gal)

templates_sn = glob.glob('sne/**/*')
templates_sn = [x for x in templates_sn if 'CVS' not in x and 'README' not in x]
templates_sn = np.array(templates_sn)

templates_dust = np.array([-0.2, 0.1])


# Truncate sn templates
#### User input goes here

In [7]:
templates_sn_trunc=np.array([x for x in templates_sn if '/II/' in x])

# compute cartesian product

In [8]:
cartesian_product_all=cartesian_product(*[templates_gal, templates_sn_trunc, templates_dust])
cartesian_product_all


array([['gal/pegase/spec_E.dat', 'sne/II/sn2005db.p86.dat', '-0.2'],
       ['gal/pegase/spec_E.dat', 'sne/II/sn2005db.p86.dat', '0.1'],
       ['gal/pegase/spec_E.dat', 'sne/II/sn1997cy.u31.dat', '-0.2'],
       ...,
       ['gal/poggianti/Sa_bianca', 'sne/II/sn2005cs.p31.dat', '0.1'],
       ['gal/poggianti/Sa_bianca', 'sne/II/sn1987K.p07.dat', '-0.2'],
       ['gal/poggianti/Sa_bianca', 'sne/II/sn1987K.p07.dat', '0.1']],
      dtype='<U32')

# make input table nicer

In [9]:
cartesian_product_all=table.Table(cartesian_product_all, names=('GALAXY', 'SN', 'DUST'))
cartesian_product_all['OBJECT']=cartesian_product_all['SN'][0]
cartesian_product_all

GALAXY,SN,DUST,OBJECT
str32,str32,str32,str23
gal/pegase/spec_E.dat,sne/II/sn2005db.p86.dat,-0.2,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn2005db.p86.dat,0.1,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn1997cy.u31.dat,-0.2,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn1997cy.u31.dat,0.1,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn2005cs.p14.dat,-0.2,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn2005cs.p14.dat,0.1,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn1987a.m84.dat,-0.2,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn1987a.m84.dat,0.1,sne/II/sn2005db.p86.dat
gal/pegase/spec_E.dat,sne/II/sn2004et.p47.dat,-0.2,sne/II/sn2005db.p86.dat


In [10]:
index_array=range(len(cartesian_product_all))

In [25]:

import time

def wrapper_fit(DATABASE):
 
    tstart=time.time()
    # read in templates
    temp_gal = np.loadtxt(DATABASE['GALAXY'])
    temp_SN = np.loadtxt(DATABASE['SN'])
 
    # object
    temp_object = np.loadtxt(DATABASE['OBJECT'])
   
    #print(time.time()-tstart)
    
    
    # read in templates
#    temp_gal = ascii.read(DATABASE['GALAXY'])
#    print(temp_gal)
    
#    temp_SN = ascii.read(DATABASE['SN'])
    #print(temp_gal,temp_SN)
    # object
#    temp_object = ascii.read(DATABASE['OBJECT'])
    #print(temp_object)    
    # interpolation
#    print(temp_gal)
    
#    temp_gal[:,1] = temp_gal[:,1]/np.median(temp_gal[:,1][(temp_gal[:,0] > 5000) & (temp_gal[:,0] < 6000)])
#    temp_SN[:,1] = temp_SN[:,1]/np.median(temp_SN[:,1][(temp_SN[:,0] > 5000) & (temp_SN[:,0] < 6000)])
#    temp_object[:,1] = temp_object[:,1]/np.median(temp_object[:,1][(temp_object[:,0] > 5000) & (temp_object[:,0] < 6000)])
    
    temp_gal_interp = interpolate.interp1d(temp_gal[:,0], temp_gal[:,1], bounds_error=False, fill_value=np.nan)
    temp_sn_interp = interpolate.interp1d(temp_SN[:,0], temp_SN[:,1], bounds_error=False, fill_value=np.nan)
    temp_object_interp = interpolate.interp1d(temp_object[:,0], temp_object[:,1], bounds_error=False, fill_value=np.nan)
    
    
    # common wavelength range
    
    lambda_min = max([temp_gal[:,0][0], temp_SN[:,0][0], temp_object[:,0][0]])
    lambda_max = min([temp_gal[:,0][-1], temp_SN[:,0][-1], temp_object[:,0][-1]])
    
    # Truncate wavelength interval
    
    lambda_new= temp_object[:,0][ (temp_object[:,0] >= lambda_min) & (temp_object[:,0] <= lambda_max) ]
#    print(lambda_new)
#    print(temp_object_interp(lambda_new))
#    print(temp_sn_interp(lambda_new))
#    print(temp_gal_interp(lambda_new))

    
    #print(lambda_min, lambda_max, lambda_new)
    
    # fit function
    
#    def func(x, b, d, c, z):
    def func(x, b, d, z):        
#        return d*temp_sn_interp(x/(z+1))*pow(10, Alam(x/(z+1))*c)+b*temp_gal_interp(x/(z+1))
        return d*temp_sn_interp(x/(z+1))+b*temp_gal_interp(x/(z+1))

    # fitting
     
    result =scipy.optimize.curve_fit(func, lambda_new, temp_object_interp(lambda_new))
    popt  = result[0]
    pcov  = result[1]
    
    # Compute chi^2
    sigma=1
    chi2 = np.sum(((temp_object_interp(lambda_new)-func(lambda_new, *popt))/sigma)**2)
    
#    return table.Table(np.array([DATABASE['GALAXY'], DATABASE['SN'], chi2, popt[0], popt[1], popt[2], np.sqrt(pcov[0,0]), np.sqrt(pcov[1,1]), np.sqrt(pcov[2,2])]), names=('GALAXY', 'SN', 'CHI2', 'CONST_GAL', 'CONST_SN', 'CONST_EXT', 'CONST_GAL_ERR', 'CONST_SN_ERR', 'CONST_EXT_ERR'), dtype=('S100', 'S100', 'f', 'f', 'f', 'f', 'f', 'f', 'f'))
    return table.Table(np.array([DATABASE['GALAXY'], DATABASE['SN'], chi2, popt[0], popt[1], 0, np.sqrt(pcov[0,0]), np.sqrt(pcov[1,1]), 0]), names=('GALAXY', 'SN', 'CHI2', 'CONST_GAL', 'CONST_SN', 'CONST_EXT', 'CONST_GAL_ERR', 'CONST_SN_ERR', 'CONST_EXT_ERR'), dtype=('S100', 'S100', 'f', 'f', 'f', 'f', 'f', 'f', 'f'))
    
#    print(result, chi2)
    

    

In [26]:
import time
tstart=time.time()
output=wrapper_fit(cartesian_product_all[index_array[0]])
time.time()-tstart

0.04594898223876953

In [13]:
output

GALAXY,SN,CHI2,CONST_GAL,CONST_SN,CONST_EXT,CONST_GAL_ERR,CONST_SN_ERR,CONST_EXT_ERR
bytes100,bytes100,float32,float32,float32,float32,float32,float32,float32
gal/pegase/spec_E.dat,sne/II/sn2005db.p86.dat,nan,1.0,1.0,0.0,inf,inf,0.0


In [14]:
if mp.cpu_count() > 1:
     number_cpu	= mp.cpu_count()/2	# is equal to number of threads x number of physical cpus, e.g. 2x4
else:
    number_cpu	= mp.cpu_count()
number_cpu
pool	= mp.Pool(processes=int(number_cpu)*2)



In [15]:
#pool = [multiprocessing.Process(target=stretch, args= (shared_arr,slice(i, i+step)),kwargs=args) for i in range (0, y, step)]
import time

tstart=time.time()
result	= pool.map(wrapper_fit, cartesian_product_all[:100])
print(time.time()-tstart)

1.781191110610962


In [16]:
result=table.vstack(result)

In [17]:
result

GALAXY,SN,CHI2,CONST_GAL,CONST_SN,CONST_EXT,CONST_GAL_ERR,CONST_SN_ERR,CONST_EXT_ERR
bytes100,bytes100,float32,float32,float32,float32,float32,float32,float32
gal/pegase/spec_E.dat,sne/II/sn2005db.p86.dat,nan,1.0,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn2005db.p86.dat,nan,1.0,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn1997cy.u31.dat,nan,1.0,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn1997cy.u31.dat,nan,1.0,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn2005cs.p14.dat,nan,1.0,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn2005cs.p14.dat,nan,1.0,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn1987a.m84.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn1987a.m84.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn2004et.p47.dat,nan,1.0,1.0,0.0,inf,inf,0.0


In [18]:
min(result['CHI2'])

nan

In [19]:
result[result['CHI2'] == min(result['CHI2'])]

GALAXY,SN,CHI2,CONST_GAL,CONST_SN,CONST_EXT,CONST_GAL_ERR,CONST_SN_ERR,CONST_EXT_ERR
bytes100,bytes100,float32,float32,float32,float32,float32,float32,float32


In [20]:
result.sort('CHI2')
result[:20]

GALAXY,SN,CHI2,CONST_GAL,CONST_SN,CONST_EXT,CONST_GAL_ERR,CONST_SN_ERR,CONST_EXT_ERR
bytes100,bytes100,float32,float32,float32,float32,float32,float32,float32
gal/pegase/spec_E.dat,sne/II/theory99em.p10.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/theory99em.p10.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn1987a.p113.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/sn1987a.p113.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/theory99em.p05.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/theory99em.p05.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/theory99em.45.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/theory99em.45.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
gal/pegase/spec_E.dat,sne/II/theory99em.p15.dat,2.7879984e+18,1.1682375e-21,1.0,0.0,inf,inf,0.0
